In [ ]:
import json
from datetime import datetime

from constants import INPUT_DIR, DATA_DIR, EVENTS_DIR
from utils import load_data

In [ ]:
print("Load event info")
event_keywords = json.load(open(f"{INPUT_DIR}/event_keyword_dict.json"))
event_times = json.load(open(f"{INPUT_DIR}/event_times.json"))

print("Load user affiliation dict")
user_affiliation = json.load(open(f"{DATA_DIR}/user_affiliation.json"))

MASS_SHOOTING_LEMMAS = "shoot|gun|kill|attack|massacre|victim"

In [ ]:
for event, keywords in event_keywords.items():
    print(event, keywords)
    event_date = datetime.fromtimestamp(event_times[event])
    event_year = event_date.year
    event_month = event_date.month
    event_day = event_date.day

    # TODO: check if last days of month or year
    reddit = load_data(DATA_DIR, year=event_year, months=[event_month], tokenize=False,
                       comp="parquet", dev=False)

    reddit = reddit[reddit.body.str.contains(keywords) & reddit.body.stry.contains(MASS_SHOOTING_LEMMAS)]
    reddit["affiliation"] = reddit.author.apply(lambda a: user_affiliation.get(a, None), meta=('affiliation', 'string'))
    reddit = reddit[reddit.affiliation.notnull()]

    reddit_df = reddit.compute()

    reddit_df.to_csv(f'{EVENTS_DIR}/{event}.csv')